### Load data

In [ ]:
import os
import pandas as pd

In [ ]:
DATA_DIR = os.path.join(os.path.realpath(''), 'data', 'homogeneous')

p2p_df = pd.read_csv(os.path.join(DATA_DIR, 'p2p.csv'))
terms_df = pd.read_csv(os.path.join(DATA_DIR, 'terms.csv'))
labels_df = pd.read_csv(os.path.join(DATA_DIR, 'labels.csv'))

In [ ]:
from graph_ml.utility.graph_representation import normalize_adj_matrix

In [ ]:
norm_adj_matrix = normalize_adj_matrix(p2p_df.to_numpy(), add_self_connection=True)
terms = terms_df.to_numpy()
labels = labels_df['Label'].to_numpy()

### Split data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train_indices, test_indices, train_labels, test_labels = train_test_split(np.arange(len(labels)), labels, test_size=0.2, random_state=0)

### GCN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers as l

from graph_ml.utility.feed_forward_layer import FeedForwardLayer
from graph_ml.gnn.gcn.gcn_layer import GCNLayer

In [ ]:
class GCNClassifier(Model):
    def __init__(self, node_features, norm_adj_matrix, num_classes):
        super().__init__()
        
        norm_adj_matrix = tf.constant(norm_adj_matrix, dtype=tf.float32)
        
        self.node_features = tf.constant(node_features, dtype=tf.float32)
        
        self.preprocessing1 = FeedForwardLayer([512])
        
        self.gcn1 = GCNLayer(norm_adj_matrix, hidden_units=[384, 256])
        self.gcn2 = GCNLayer(norm_adj_matrix, hidden_units=[256, 192])
        self.gcn3 = GCNLayer(norm_adj_matrix, hidden_units=[192, 96])
        
        self.postprocessing1 = FeedForwardLayer([64])

        self.logits1 = l.Dense(num_classes)
        self.softmax1 = l.Softmax()
        
    def call(self, indices):
        features = self.preprocessing1(self.node_features)
        
        features = self.gcn1(features)
        features = self.gcn2(features)
        features = self.gcn3(features)
        
        features = tf.gather(features, indices)
        
        features = self.postprocessing1(features)
        logits = self.logits1(features)
        return self.softmax1(logits)

### Classify nodes

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [ ]:
num_classes = len(set(labels))

model = GCNClassifier(terms, norm_adj_matrix, num_classes)
model.compile(optimizer=Adam(1e-4), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy(name="accuracy")])
_ = model.fit(x=train_indices, y=labels[train_indices], batch_size=500, epochs=20, validation_split=0.2, verbose=0)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
true_labels = labels[test_indices]
predicted_labels = model(test_indices).numpy().argmax(axis=1)
print(classification_report(true_labels, predicted_labels))